In [1]:
from pyomo.environ import *
import os
model = ConcreteModel()
solverexe = "gurobi"
# dirsolver = r"C:\Users\ch9fod\Documents\GitHub\ED\solvers"
# datafile = "data1.dat"

In [2]:
#set
model.Bus = RangeSet(4, doc='Buses')
model.G = RangeSet(3, doc='Generators')

In [3]:
#parameters
model.a = Param(model.G, initialize={1:20 , 2:25 , 3:40}, doc='Parameter a')
model.b = Param(model.G, initialize={1:0.05 , 2:0.10 , 3:0.20}, doc='Parameter b')
model.Pmin = Param(model.G, initialize={1:20 , 2:20 , 3:30}, doc='Parameter Pmin')
model.Pmax = Param(model.G, initialize={1:400 , 2:300 , 3:250}, doc='Parameter Pmax')
# model.d = Param(model.G, initialize={1:300 , 2:300 , 3:100}, doc='Parameter d')
# model.e = Param(model.G, initialize={1:200 , 2:200 , 3:100}, doc='Parameter e')
# model.f = Param(model.G, initialize={1:0.01 , 2:0.01 , 3:0.005}, doc='Parameter f')
#lone parameter, constant
model.D = Param(initialize=600, doc='Load')
model.PFmax = Param(initialize=300, doc='Max Power Flow')
model.PFmin = Param(initialize=-300, doc='Min Power Flow')

In [4]:
#variables
model.P = Var(model.G, doc='Generation per Generator')
model.Angles = Var(model.Bus, doc='Angles of Buses')
model.Angles[1] = 0;

In [ ]:
#constraints
def maxp(model,i):
    return model.P[i] <= model.Pmax[i]
model.maxprod = Constraint(model.G, rule = maxp)

def minp(model,i):
    return model.P[i] >= model.Pmin[i]
model.minprod = Constraint(model.G, rule = minp)

def demand_r(model):
    return model.D == sum(model.P[i] for i in model.G)
model.demand = Constraint(model.G, rule = demand_r)

In [ ]:
#objective
def cost_rule(model):
    return sum(model.a[i]*model.P[i] + 
               0.5*model.b[i]*model.P[i]**2 
               for i in model.G)
#default is to minimize        
model.OBJ = Objective(rule=cost_rule) 

In [ ]:
if solverexe == "gurobi":
    solver = SolverFactory(solverexe)   
else:
    solver = SolverFactory(solverexe, 
                           executable=os.path.join(dirsolver, solverexe))
instance = model.create_instance()
instance.dual = Suffix(direction=Suffix.IMPORT)
results = solver.solve(instance)

In [ ]:
instance.display()
instance.pprint()

In [ ]:
value(instance.OBJ)

In [ ]:
p = [0, 0, 0 ,0]
for i in range(3):
    p[i+1] = value(instance.P[i+1])

In [ ]:
import locale
locale.setlocale(locale.LC_ALL, '')
print ("Total Emissions (tonCO2/MWh)")
x = y = 0
for i in range(3):
    y = (instance.d[i+1] + instance.e[i+1]*p[i+1] + 
        instance.f[i+1]*p[i+1]**2)
    x = x + y
    print(i+1, locale.format("%d", y, grouping=True))
print("Total",locale.format("%d", x, grouping=True)) 


In [ ]:
print ("Duals")
from pyomo.core import Constraint
for c in instance.component_objects(Constraint, active=True):
    print ("   Constraint",c)
    cobject = getattr(instance, str(c))
    for index in cobject:
        print ("      ", index, instance.dual[cobject[index]])